In [1]:
#关于文档的一个数据分类
import pandas as pd
import seaborn as sns  #用于绘制热图的工具包
from scipy.cluster import hierarchy  #用于进行层次聚类，话层次聚类图的工具包
from scipy import cluster   
import matplotlib.pyplot as plt
from sklearn import decomposition as skldec #用于主成分分析降维的包
import numpy as np
import math
from pandas import Series,DataFrame
from matplotlib.ticker import  MultipleLocator

In [2]:
data_allvc = pd.read_excel('E:/vcdata/vcdata(91-17).xlsx') ### Raw data for venture capital
data_allvc.loc[data_allvc['投资金额(M)'].str.contains('数十万')==True,'投资金额(M)']='0.1'
data_allvc.loc[data_allvc['投资金额(M)'].str.contains('数百万')==True,'投资金额(M)']='1'
data_allvc.loc[data_allvc['投资金额(M)'].str.contains('数千万')==True,'投资金额(M)']='10'
data_allvc.loc[data_allvc['投资金额(M)'].str.contains('数亿')==True,'投资金额(M)']='100'
data_allvc.loc[data_allvc['投资金额(M)'].str.contains('数十亿')==True,'投资金额(M)']='1000'
data_allvc.loc[data_allvc['投资金额(M)'].str.contains('<')==True,'投资金额(M)']='0.005'
data_allvc.loc[data_allvc['投资金额(M)'].str.contains('-')==True,'投资金额(M)']='-1'
data_allvc.loc[data_allvc['投资金额(M)'].isin(['不披露'])==True,'投资金额(M)']='0' ###不批露的也记为0，0表示数据未知

In [3]:
dic_money={'AUD':4.8042,'CAD':5.0319,'EUR':7.5691,'GBP':8.7794,'HKD':0.8559,'JPY':0.06042,'KRW':0.005893,
           'NOK':0.7900,'NTD':4.128,'RMB':1,'SGD':4.9613,'TWD':0.2177,'USD':6.7126,'N/A':1
          }
data_allvc['change']=data_allvc['投资币种'].map(dic_money)
data_allvc['change']=data_allvc['change'].fillna('1')
data_allvc['投资金额(M)'] = data_allvc['投资金额(M)'].astype(float)
data_allvc['change'] = data_allvc['change'].astype(float)
amount = data_allvc['投资金额(M)'].values
change = data_allvc['change'].values
amountM = []
for i in range(data_allvc.shape[0]):    
    if amount[i] < 0 or change[i] < 0:
        amountM.append('--')
    else:
        amountM.append(amount[i] * change[i])
data_allvc['投资金额(RMB)'] = amountM
# data['change']=data['invest_currency'].map(dic_money)

In [4]:
######## 记录每个城市每年总投资金额、总投资次数，有投资的次数，无投资的次数
def getstr(s): ### 投资时间统一为yyyy
    return str(s)[:4]##
data_allvc['year'] = data_allvc['投资时间'].map(getstr)

In [5]:
#得到各省的市
def get_areas(a):
    a=list(a.split('|'))
    for i in range(len(a)):
        if a[i].find('市')!=-1 or a[i].find('州')!=-1 or a[i].find('地区')!=-1 or a[i].find('盟')!=-1 or a[i].find('香港')!=-1 or a[i].find('澳门')!=-1 or a[i].find('台湾')!=-1 or a[i].find('自治县')!=-1:
            return(a[i])       
data_allvc['city']=data_allvc['实际地区'].map(get_areas)
data_allvc['city']=data_allvc['city'].fillna('--')
#data.new_area

In [7]:
industry_date=pd.read_excel('E:/文档系列/python 系列/数据处理过滤/清科产业对应国标分级.xlsx') ###请客和产业对比
#处理两个表中的空数据
industry_date.fillna("--", inplace=True)
# datw.fillna("--", inplace=True)
#分别写出行业（清科）和三级代码、三级产业名称、二级代码、二级名称、一级字母代码、一级名称、一二三产业对应关系
Tertiary_code=industry_date.set_index("行业(清科)").to_dict()['三级代码']
Tertiary_Industry=industry_date.set_index("行业(清科)").to_dict()['三级产业名称']
Secondary_code=industry_date.set_index("行业(清科)").to_dict()['二级代码']
Secondary_Industry=industry_date.set_index("行业(清科)").to_dict()['二级名称']
Primary_code=industry_date.set_index("行业(清科)").to_dict()['一级字母代码']
Primary_Industry=industry_date.set_index("行业(清科)").to_dict()['一级名称']
Correspondence=industry_date.set_index("行业(清科)").to_dict()['一二三产业']
City_city=data_allvc.set_index("city").to_dict()['实际地区']
#d系列函数分别是为了找到'行业(清科)'和一二三产业、代号的对应关系
def d1(s):
    end=Tertiary_code[s]
    return(end)
def d2(s):
    end=Tertiary_Industry[s]
    return(end)
def d3(s):
    end=Secondary_code[s]
    return(end)
def d4(s):
    end=Secondary_Industry[s]
    return(end)
def d5(s):
    end=Primary_code[s]
    return(end)
def d6(s):
    end=Primary_Industry[s]
    return(end)
def d7(s):
    end=Correspondence[s]
    return(end)
data_allvc['三级代码']=data_allvc['行业(清科)'].map(d1)
data_allvc['三级产业名称']=data_allvc['行业(清科)'].map(d2)
data_allvc['二级代码']=data_allvc['行业(清科)'].map(d3)
data_allvc['二级名称']=data_allvc['行业(清科)'].map(d4)
data_allvc['一级字母代码']=data_allvc['行业(清科)'].map(d5)
data_allvc['一级名称']=data_allvc['行业(清科)'].map(d6)
data_allvc['一二三产业']=data_allvc['行业(清科)'].map(d7)
# data_allvc['City']=data_allvc['city'].map(citys)

In [6]:
data_allvc.to_excel('E:/vcdata/vcdata(91-17-new2).xlsx') ###处理后的数据进行保存

In [8]:
len(data_allvc['city'].drop_duplicates().values)

339